In [1]:
from siphon.catalog import TDSCatalog
import xarray as xr
import rioxarray as rxr

from google.cloud import storage
from datetime import datetime, date, timedelta
import os

In [2]:
os.environ['WEST'] = '90.0'
os.environ['EAST'] = '150.0'
os.environ['SOUTH'] = '-15.0'
os.environ['NORTH'] = '15.0'
os.environ['VARIABLE'] = 'daymean_cnc_PM2_5'

In [3]:
# Observation time
obs = date.today()
obs_time = datetime(obs.year, obs.month, obs.day, 12, 30)

URL = 'http://silam.fmi.fi/thredds/catalog/dailysilam_glob_v5_7_1/catalog.xml'
catalog = TDSCatalog(URL)

In [4]:
# NetCDF Subset Service
ncss = catalog.datasets[0].subset()

# Query boundary box and variable
variable = os.environ.get('VARIABLE')
west = float(os.environ.get('WEST'))
east = float(os.environ.get('EAST'))
south = float(os.environ.get('SOUTH'))
north = float(os.environ.get('NORTH'))

# Create subset query
query = ncss.query()
query.variables(variable).time(obs_time)
query.lonlat_box(west=west, east=east, south=south, north=north)
data = xr.backends.NetCDF4DataStore(ncss.get_data(query))

ds = xr.open_dataset(data).isel(time = 0) \
    .isel(hybrid = 0, drop = True)

In [5]:
ds

<xarray.Dataset>
Dimensions:            (lat: 151, lon: 301)
Coordinates:
    time_run           datetime64[ns] ...
    time               datetime64[ns] 2021-07-18T12:30:00
  * lat                (lat) float32 -15.0 -14.8 -14.6 -14.4 ... 14.6 14.8 15.0
  * lon                (lon) float32 90.0 90.2 90.4 90.6 ... 149.6 149.8 150.0
Data variables:
    daymean_cnc_PM2_5  (lat, lon) float32 ...
Attributes:
    Conventions:         CF-1.4
    cdm_data_type:       GRID
    featureType:         GRID
    location:            Proto fmrc:daily_silam_glob_v5_7_1
    history:             FMRC Best Dataset
    History:             Translated to CF-1.0 Conventions by Netcdf-Java CDM ...
    geospatial_lat_min:  -15.0
    geospatial_lat_max:  15.0
    geospatial_lon_min:  90.00001525878906
    geospatial_lon_max:  150.00001525878906

In [6]:
date_str = obs.strftime('%Y%m%d')
fname = f'pm25_{date_str}.tif'

concentration = ds[variable]
concentration.rio.set_spatial_dims('lon', 'lat', inplace=True)
concentration.rio.set_crs(4326, inplace=True)
concentration.rio.to_raster(fname)

In [7]:
os.system(f'rio cogeo create {fname} {fname}')

client = storage.Client.from_service_account_json('neonet-sentinel-privatekey.json')
bucket = client.bucket('silam-neonet-rasters')
blob = bucket.blob(fname)
blob.upload_from_filename(fname)

os.remove(fname)

/home/josef/miniconda3/envs/geo/lib/python3.9/site-packages/rio_cogeo/cogeo.py:189: IncompatibleBlockRasterSize: Block Size are bigger than raster sizes. Setting blocksize to 128
  warnings.warn(
Reading input: /home/josef/Documents/Git/GeospatialPython/SILAM/pm25_20210718.tif
Adding overviews...
Updating dataset tags...
Writing output to: /home/josef/Documents/Git/GeospatialPython/SILAM/pm25_20210718.tif
